In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
!pip install mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.0/700.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
  

In [6]:
import dagshub
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'lmamu21' 
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8bc574422c1ba5ebd3c7e16e00460a8560803a94'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/lmamu21/fraud-detection.mlflow'

dagshub.init(repo_owner='lmamu21', repo_name='fraud-detection', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d2b58bb9-f109-4b14-8eab-a8c2cd244035&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a2b043a8d5ad2438f07c4d5544f4721ccb0ec0f164b001051c3f921a27a1b656




Output()

Accessing as lmamu21

Initialized MLflow to track repo "lmamu21/fraud-detection"

Repository lmamu21/fraud-detection initialized!

In [114]:
test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
test = pd.merge(test_transaction, test_identity, on="TransactionID", how="left")

In [115]:
submission = pd.DataFrame()
submission["TransactionID"] = test["TransactionID"]

In [116]:
import mlflow
import joblib
import pandas as pd


RUN_ID = "c180072902cb469fbcc97edc15eb0e75"
MODEL_URI = f"runs:/{RUN_ID}/random_forest_model_undersampled_low"
PREPROCESSOR_PATH = f"./mlruns/0/{RUN_ID}/artifacts/pre_undersampled_low.pkl"

In [117]:
import mlflow.sklearn

model = mlflow.sklearn.load_model(MODEL_URI)


In [118]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 null_threshold=0.6, 
                 encoding_threshold=7, 
                 sampling_strategy='none', 
                 target_ratio=0.5, 
                 l1_regularization=False, 
                 l1_C=0.01):
        self.null_threshold = null_threshold
        self.encoding_threshold = encoding_threshold
        self.sampling_strategy = sampling_strategy
        self.target_ratio = target_ratio
        self.l1_regularization = l1_regularization
        self.l1_C = l1_C
        
    def fit(self, X, y=None):
        X = X.copy()
        
        # 1. Identify columns to drop
        null_frac = X.isnull().mean()
        self.cols_to_drop_ = null_frac[null_frac > self.null_threshold].index.tolist()
        X = X.drop(columns=self.cols_to_drop_, errors='ignore')
        
        # 2. Update cat_cols and num_cols after dropping
        self.cat_cols_ = X.select_dtypes(include=['object', 'category']).columns.tolist()
        self.num_cols_ = [col for col in X.columns if col not in self.cat_cols_]
        
        # 3. Save fill values
        self.fill_values_ = {}
        for col in self.num_cols_:
            self.fill_values_[col] = X[col].median()
        for col in self.cat_cols_:
            self.fill_values_[col] = X[col].mode(dropna=True)[0]
        
        # 4. Identify columns to apply WOE
        self.onehot_cols_ = []
        self.woe_cols_ = []
       
        for col in self.cat_cols_:
            if X[col].nunique() <= self.encoding_threshold:
                self.onehot_cols_.append(col)
            else:
                self.woe_cols_.append(col)
        
        # 5. Compute WOE mappings for WOE columns
        self.woe_maps_ = {}
        if y is not None:
            for col in self.woe_cols_:
                self.woe_maps_[col] = self._compute_woe(X[col], y)

        # 6. L1 feature selection
        if self.l1_regularization and y is not None:
            X_basic = self._basic_clean(X)
            model = LogisticRegression(penalty='l1', solver='liblinear', C=self.l1_C, max_iter=1000)
            model.fit(X_basic, y)
            non_zero_coef = model.coef_[0] != 0
            self.selected_features_ = X_basic.columns[non_zero_coef].tolist()
        else:
            self.selected_features_ = None

        return self

    def transform(self, X):
        X = X.copy()
        
        # 1. Drop bad columns
        X = X.drop(columns=self.cols_to_drop_, errors='ignore')
        
        # 2. Fill missing values
        for col, fill_value in self.fill_values_.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)
        
        # 3. Apply WOE encoding for selected columns
        for col in self.woe_cols_:
            if col in X.columns:
                X[col] = X[col].map(self.woe_maps_.get(col, {})).fillna(0)

        # 4. Apply One-Hot encoding for other columns
        X = pd.get_dummies(X, columns=self.onehot_cols_, drop_first=True)
        
        # 5. If L1 selection, keep only selected features
        if self.selected_features_ is not None:
            for feature in self.selected_features_:
                if feature not in X.columns:
                    X[feature] = 0
            X = X[self.selected_features_]
        
        return X

    def fit_resample(self, X, y):
        """Optional resampling after cleaning"""
        X_clean = self.fit(X, y).transform(X)
        
        if self.sampling_strategy == 'undersample':
            fraud = X_clean[y == 1]
            legit = X_clean[y == 0]
            legit_downsampled = resample(legit, replace=False, 
                                         n_samples=int(len(fraud) / self.target_ratio - len(fraud)), 
                                         random_state=42)
            X_resampled = pd.concat([fraud, legit_downsampled])
            y_resampled = np.array([1]*len(fraud) + [0]*len(legit_downsampled))
        
        elif self.sampling_strategy == 'undersample':
            fraud = X_clean[y == 1]
            legit = X_clean[y == 0]
            fraud_upsampled = resample(fraud, replace=True, 
                                       n_samples=int(len(legit) * self.target_ratio / (1 - self.target_ratio)), 
                                       random_state=42)
            X_resampled = pd.concat([fraud_upsampled, legit])
            y_resampled = np.array([1]*len(fraud_upsampled) + [0]*len(legit))
        
        else:
            X_resampled = X_clean
            y_resampled = y
        
        return X_resampled, y_resampled
    
    def _basic_clean(self, X):
        X = X.drop(columns=self.cols_to_drop_, errors='ignore')
        for col, fill_value in self.fill_values_.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)
        X = pd.get_dummies(X, columns=self.cat_cols_, drop_first=True)
        return X

    def _compute_woe(self, series, y):
        df = pd.DataFrame({'feature': series, 'target': y})
        grouped = df.groupby('feature')['target']
        event = grouped.sum()
        non_event = grouped.count() - event
        event_rate = (event + 0.5) / event.sum()
        non_event_rate = (non_event + 0.5) / non_event.sum()
        woe = np.log(event_rate / non_event_rate)
        return woe.to_dict()


In [119]:
import shutil
PREPROCESSOR_LOG_RUN_ID = "e0a57499a4dc4ebbb694b91eac55d6d1";
mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{PREPROCESSOR_LOG_RUN_ID}/pre_undersampled_low.pkl", dst_path=".")
preprocessor = joblib.load("pre_undersampled_low.pkl")

In [120]:
X_test = test.drop(columns=["TransactionID"], errors="ignore")
X_test_clean = preprocessor.transform(X_test)

In [121]:
# sorry for hard codding
train_columns = ["TransactionID", "TransactionAmt", "card1", "card2", "card3", "addr1", "P_emaildomain", "C1", "C5", "D1", "D3", "D4", "D5", "D10", "V302"]

In [122]:
# Make sure columns match the training columns (optional):
X_test_clean = X_test_clean.reindex(columns=train_columns, fill_value=0)

# --- Predict ---
y_pred_proba = model.predict_proba(X_test_clean)[:, 1]


# --- Write submission ---
submission["isFraud"] = y_pred_proba
submission.to_csv("submission.csv", index=False)